## 第4回レポート課題

### Softmax with lossレイヤーの実装

In [8]:
import numpy as np
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    #教師データがone-hot-vectorの場合, 正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [9]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None 
        self.y = None #softmaxの出力
        self.t = None #教師データ
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx =  (self.y - self.t) / batch_size 
        #yi - tiを計算してバッチサイズで割ってデータ一つあたりにする 
        return dx

 ここでReluレイヤとAffineレイヤを実装した. 

In [32]:
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        return out
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None
        #重みとバイアスパラメータの微分の保持
        self.dW = None
        self.db = None
    def forward(self, x):
        #テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = dx.reshape(*self.original_x_shape)
        return dx

以上のことから, レイヤを用いて2層ニューラルネットワークの実装をした.

In [33]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        #重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        #レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis = 1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 勾配確認の実装

In [35]:
from dataset.mnist import load_mnist
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]
# 数値微分
grad_numerical = network.numerical_gradient(x_batch, t_batch)
# Backward
grad_backprop = network.gradient(x_batch, t_batch)
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:0.00048620383206118774
b1:0.0031511505403730698
W2:1.1712256359805007e-12
b2:1.2034816893047307e-10


誤差逆伝播法では数値微分に比べるとミスが起こりやすい. 第一層の時点では誤差がおよそ$10^{-4}$であり, この時点では勾配は数値微分に比べると大きいといえる. しかし, 第二層では重みの勾配の差がおよそ$10^{-12}$, バイアスの勾配の差がおよそ$10^{-10}$とほとんど近い値といえる. 前回考えた処理速度も考慮すると, 誤差逆伝播法を用いた方が良いことが分かる.

### Two layer netの学習

In [37]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learing_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #  誤差逆伝播法によって勾配を求める
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learing_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.12363333333333333 0.1256
0.8983333333333333 0.9035
0.9131666666666667 0.918
0.9159833333333334 0.9168
0.9197 0.9232
0.92115 0.92
0.9243166666666667 0.923
0.9222166666666667 0.9169
0.9220166666666667 0.9199
0.9228833333333334 0.9209
0.92535 0.9219
0.9170166666666667 0.9174
0.9209166666666667 0.9203
0.92025 0.9167
0.9152333333333333 0.9143
0.9093333333333333 0.9072
0.91555 0.9164


初回の出力値のみ12%とかなり低い. しかし, 二回目以降の出力はおよそ90%を超えており, 精度としては問題ないと考えられる. また, 精度の最大値が最後の方ではなく, 中盤の出力値であることから, 少し過学習が起きていると考えた. ただ, 精度の誤差はおよそ1%ほどであるため, 問題はないと判断した.

### 感想
レイヤの実装は導出が少し難しい部分もあったが, おおむね理解しやすかった. また, 各レイヤを順番付きディクショナリに保持することで, 順伝播の処理を行えるということが非常に面白く感服した.

### 参考文献
斎藤 康毅　『ゼロから作るDeep Learning』, 2019, オライリー・ジャパン, p.123-164